In [1]:
GRAMMAR = r"""
    %import common.LETTER
    %import common.DIGIT
    %import common.INT
    %import common.NUMBER
    %import common.WORD
    %import common.CNAME
    %import common.ESCAPED_STRING

    %import common.WS
    %ignore WS
    %import common.SH_COMMENT
    %ignore SH_COMMENT

    program         : stmt_lst
    stmt_lst        : statement+

    statement       : expression | return_stmt | if_stmt | for_stmt 
    return_stmt     : "return" expression ~ 1 -> return_statement
    if_stmt         : "if" "(" expression ")" stmt_lst "end" -> if_statement
    for_stmt        : "for" "(" expression ";" expression ";" expression ")" stmt_lst "end" -> for_statement

    expression      : assignment | logic_or
    assignment      : var_assign | arr_assign
    var_assign      : var "=" expression -> var_assignment
    arr_assign      : var "[" expression "]" "=" expression -> array_assignment
    logic_or        : logic_and 
                    | logic_or "or" logic_or -> or
    logic_and       : equality 
                    | logic_and "and" logic_and -> and
    equality        : comparison 
                    | equality "!=" equality -> neq
                    | equality "==" equality -> eeq
    comparison      : term 
                    | comparison ">" comparison -> ge
                    | comparison "<" comparison -> le
                    | comparison ">=" comparison -> geq
                    | comparison "<=" comparison -> leq
    term            : factor 
                    | term "+" term -> add
                    | term "-" term -> sub
    factor          : unary 
                    | factor "*" factor -> mul
                    | factor "/" factor -> div
    unary           : call
                    | "!" unary -> not
                    | "-" unary -> neg
    call            : atom 
                    | atom ( "(" arguments ")" )* 
    atom            : "true" -> true 
                    | "false" -> false 
                    | NUMBER -> number
                    | ESCAPED_STRING -> string
                    | var
                    | var "[" expression "]" -> array_access
                    | "[" expression ( "," expression )* "]" -> array
                    | "(" expression ")"
    var             : CNAME
    arguments       : expression ( "," expression )*
"""

In [2]:
from lark import Lark

parser = Lark(
    GRAMMAR,
    start="program",
    parser="lalr"
)

#if_stmt         : "if" "(" expression ")" statement+ ("else" statement+)? "end" -> if_statement
#for_stmt        : "for" "(" expression ";" expression ";" expression ")" stmt_lst "end" -> for_statement

code = """
for (i = 0; i < 3; i = i + 1)
    print(i)
end
"""

tree = parser.parse(code)
tmp = tree.children[0].children[0].children[0]

print(tmp.pretty())
print(tmp)

print(tmp.data)
for child in tmp.children:
    print(child)
    print(child.children[0].value)

#print(tmp.children[2])
#print(tmp.children[0].data)
#print(tmp.children[0].children[0].value)

for_statement
  expression
    assignment
      var_assignment
        var	i
        expression
          logic_or
            logic_and
              equality
                comparison
                  term
                    factor
                      unary
                        call
                          number	0
  expression
    logic_or
      logic_and
        equality
          le
            comparison
              term
                factor
                  unary
                    call
                      atom
                        var	i
            comparison
              term
                factor
                  unary
                    call
                      number	3
  expression
    assignment
      var_assignment
        var	i
        expression
          logic_or
            logic_and
              equality
                comparison
                  add
                    term
                      factor
                        unary
    

AttributeError: 'Tree' object has no attribute 'value'

In [90]:
import random
import z3


def my_max(vs):
  m = vs[0]
  for v in vs[1:]:
    m = z3.If(v > m, v, m)
  return m

def opt_mis(arr: list[z3.ArithRef]) -> int:
    n = len(arr)
    dp = [0] * (n+1)
    dp[1] = arr[0]

    for i in range(2, n+1):
        dp[i] = max(dp[i-1], dp[i-2] + arr[i-1])
    return dp[n]

def mis(arr: list[z3.ArithRef], s) -> int:
    n = len(arr)
    o = opt_mis(arr)
    j = 1

    def aux_mis(n_args = 1):
        dp = z3.K(z3.IntSort(), 0)
        dp = z3.Store(dp, 1, arr[0])

        x = [z3.Int(f'x_{i}') for i in range(1, 3)]
        s.add([x[i] >= -3 for i in range(len(x))])

        a = [z3.Int(f'a_{i}') for i in range(1, 3)]
        b = [z3.Int(f'b_{i}') for i in range(1, 3)]
        s.add([a[i] >= -3 for i in range(len(a))])
        s.add([a[i] <= 3 for i in range(len(a))])
        s.add([b[i] >= -3 for i in range(len(b))])
        s.add([b[i] <= 3 for i in range(len(b))])

        for i in range(2, n+1):
            #args = 
            dp = z3.Store(dp, i, my_max([a[0]* dp[i-x[0]] + b[0]*arr[i-1], a[1]*dp[i-x[1]] + b[1]*arr[i-1]]))
        return dp[n]

    s.push()
    s.add(aux_mis(j) == o)
    while s.check() == z3.unsat and j < 4:
        j += 1
        s.pop()
        s.push()
        s.add(aux_mis(j) == o)

    if j < 4 and s.check() == z3.sat:
        return z3.sat
    else:
        return z3.unsat

s = z3.Solver()

lst = []
arrs = []
ntests = 5
n = 4
A = z3.IntVector('A', n)
for i in range(ntests):
    arr = [random.randint(0, 10) for _ in range(n)]
    arrs.append(arr)
    test = z3.Implies(A == arr, mis(arr, s) == z3.sat)
    lst.append(test)
tests = z3.And(lst)

s.add(
    z3.ForAll(A, tests)
)

if s.check() == z3.sat:
    m = s.model()
    print(m)

    decs = m.decls()
    val_x = []
    val_a = []
    val_b = []
    for dec in decs:
        match dec.name()[0]:
            case 'x':
                val_x.append(dec)
            case 'a':
                val_a.append(dec)
            case 'b':
                val_b.append(dec)

    val_x.sort(key=lambda x: int(x.name().split('_')[1]))
    val_x = [m[x] for x in val_x]
    val_a.sort(key=lambda x: int(x.name().split('_')[1]))
    val_a = [m[x] for x in val_a]
    val_b.sort(key=lambda x: int(x.name().split('_')[1]))
    val_b = [m[x] for x in val_b]

    hole1 = f"dp[i-{val_x[0]}]" if val_a[0] == 1 else (f"{val_a[0]}dp[i-{val_x[0]}]" if val_a[0] != 0 else f"")
    match val_b[0]:
        case 1:
            hole1 += f" + val"
        case 0:
            hole1 += f""
        case _:
            hole1 += f"+ {val_b[0]}val"
    
    hole2 = f"dp[i-{val_x[1]}]" if val_a[1] == 1 else (f"{val_a[1]}dp[i-{val_x[1]}]" if val_a[1] != 0 else f"")
    match val_b[1]:
        case 1:
            hole2 += f" + val"
        case 0:
            hole2 += f""
        case _:
            hole2 += f"+ {val_b[1]}val"

    print(f"Synthesized Expression: max({hole1}, {hole2})")

    #for i in range(ntests):
        #print(f"arr = {arrs[i]}, opt = {opt(arrs[i])}")
        #print(f"mis = {s.model()}")


[x = 2,
 v = Store(Store(Store(K(Int, 6), 3, 4), 4, 5), 2, 2),
 y = 1]


In [2]:
import random
import z3

def my_max(vs):
  m = vs[0]
  for v in vs[1:]:
    m = z3.If(v > m, v, m)
  return m

def opt_mis(arr: list[z3.ArithRef]) -> int:
    n = len(arr)
    dp = [0] * (n+1)
    dp[1] = arr[0]
    #dp[2] = arr[1]

    for i in range(2, n+1):
        dp[i] = max(dp[i-1], dp[i-2] + arr[i-1])
    return dp[n]

def mis(arr: list[z3.ArithRef], s, l=5, o = None) -> int:
    n = len(arr)
    if o is None:
        o = opt_mis(arr)
    
    try:
        j = 1 #len(s.model().decls()) // 3
    except z3.Z3Exception:
        j = 1

    def aux_mis(n_args = 1):
        dp = z3.K(z3.IntSort(), 0)
        dp = z3.Store(dp, 1, arr[0])
        #dp = z3.Store(dp, 2, arr[1])

        x = [z3.Int(f'x_{i}') for i in range(n_args)]
        s.add([x[i] >= 0 for i in range(len(x))])
        s.add([x[i] < n for i in range(len(x))])
        z3.Distinct(*x)

        a = [z3.Int(f'a_{i}') for i in range(n_args)]
        b = [z3.Int(f'b_{i}') for i in range(n_args)]
        s.add([a[i] >= -3 for i in range(len(a))])
        s.add([a[i] <= 3 for i in range(len(a))])
        s.add([b[i] >= -3 for i in range(len(b))])
        s.add([b[i] <= 3 for i in range(len(b))])

        def get_expr(i, val):
            args = []
            for k in range(n_args):
                args.append(a[k]*dp[i-x[k]] + b[k]*val)
            return args

        for i in range(2, n+1):
            args = get_expr(i, arr[i-1])
            dp = z3.Store(dp, i, my_max(args))
        print(dp[n])
        return dp[n]

    s.push()
    s.add(aux_mis(j) == o)
    while j < 4 and s.check() == z3.unsat:
        j += 1
        s.pop()
        s.push()
        s.add(aux_mis(j) == o)

    if j <= l and s.check() == z3.sat:
        #print(s.model())
        return z3.sat
    else:
        return z3.unsat

s = z3.Solver()

lst = []
arrs = []
ntests = 10
n = 5
A = z3.IntVector('A', n)
for i in range(ntests):
    arr = [random.randint(0, 10) for _ in range(n)]
    arrs.append(arr)
    test = z3.Implies(A == arr, mis(arr, s, l = 4) == z3.sat)
    lst.append(test)
tests = z3.And(lst)

s.add(
    z3.ForAll(A, tests)
)

if s.check() == z3.sat:
    m = s.model()
    #print(m)

    decs = m.decls()
    val_x = []
    val_a = []
    val_b = []
    for dec in decs:
        match dec.name()[0]:
            case 'x':
                val_x.append(dec)
            case 'a':
                val_a.append(dec)
            case 'b':
                val_b.append(dec)

    val_x.sort(key=lambda x: int(x.name().split('_')[1]))
    val_x = [m[x] for x in val_x]
    val_a.sort(key=lambda x: int(x.name().split('_')[1]))
    val_a = [m[x] for x in val_a]
    val_b.sort(key=lambda x: int(x.name().split('_')[1]))
    val_b = [m[x] for x in val_b]

    holes = []
    for i in range(len(val_x)):
        hole = ""
        match val_a[i]:
            case 1:
                hole += f"dp[i-{val_x[i]}]"
            case 0:
                hole += f""
            case _:
                hole += f"{val_a[i]}dp[i-{val_x[i]}]"
        match val_b[i]:
            case 1:
                hole += f" + val"
            case 0:
                hole += f""
            case _:
                hole += f" + {val_b[i]}val"
        holes.append(hole)

    print(f"Synthesized recurrence: max({', '.join(holes)})")
else:
    print("Failed to synthesize recurrence")


Store(Store(Store(Store(Store(K(Int, 0), 1, 4),
                        2,
                        a_0*Store(K(Int, 0), 1, 4)[2 - x_0] +
                        b_0*2),
                  3,
                  a_0*
                  Store(Store(K(Int, 0), 1, 4),
                        2,
                        a_0*Store(K(Int, 0), 1, 4)[2 - x_0] +
                        b_0*2)[3 - x_0] +
                  b_0*2),
            4,
            a_0*
            Store(Store(Store(K(Int, 0), 1, 4),
                        2,
                        a_0*Store(K(Int, 0), 1, 4)[2 - x_0] +
                        b_0*2),
                  3,
                  a_0*
                  Store(Store(K(Int, 0), 1, 4),
                        2,
                        a_0*Store(K(Int, 0), 1, 4)[2 - x_0] +
                        b_0*2)[3 - x_0] +
                  b_0*2)[4 - x_0] +
            b_0*9),
      5,
      a_0*
      Store(Store(Store(Store(K(Int, 0), 1, 4),
                        2,
       

In [7]:
def opt_mcs(nums: list[z3.ArithRef]) -> int:
    dp = [[0]*len(nums) for i in range(2)]
    dp[0][0], dp[1][0] = nums[0], nums[0]
    for i in range(1, len(nums)):
        dp[1][i] = max(nums[i], nums[i] + dp[1][i-1])
        dp[0][i] = max(dp[0][i-1], dp[1][i])
    return dp[0][-1]

def mcs(arr: list[z3.ArithRef], s) -> int:
    n = len(arr)
    o = opt_mcs(arr)

    dp = z3.K(z3.IntSort(), 0)
    dp = z3.Store(dp, 1, arr[0])
    dp_aux = z3.K(z3.IntSort(), 0)
    dp_aux = z3.Store(dp_aux, 1, arr[0])

    for i in range(2, n+1):
        val = arr[i-1]
        dp_aux = z3.Store(dp_aux, i, my_max([val, dp_aux[i-x] + val]))
        dp = z3.Store(dp, i, my_max([dp[i-y],  dp_aux[i-z]]))
    
    s.add(dp[n] == o)
    if s.check() == z3.sat:
        #s.pop()
        return z3.sat
    
def my_max(vs):
  m = vs[0]
  for v in vs[1:]:
    m = z3.If(v > m, v, m)
  return m
    
s = z3.Solver()

x = z3.Int('x')
y = z3.Int('y')
z = z3.Int('z')

n = 5
A = z3.IntVector('A', n)
#s.add(x >= 0)
#s.add(y >= 0)

lst = []
arrs = []
ntests = 4
for i in range(ntests):
    arr = [random.randint(-10, 10) for _ in range(n)]
    arrs.append(arr)
    test = z3.Implies(A == arr, mcs(arr, s) == z3.sat)
    lst.append(test)
tests = z3.And(lst)

s.add(
    z3.ForAll(A, tests)
)

if s.check() == z3.sat:
    print(f"x = {s.model()[x]}")
    print(f"y = {s.model()[y]}")
    print(f"z = {s.model()[z]}")

NameError: name 'random' is not defined

In [173]:
import random
import z3

def opt_mis(arr: list[z3.ArithRef]) -> int:
    n = len(arr)
    dp = [0] * (n+1)
    dp[1] = arr[0]

    for i in range(2, n+1):
        dp[i] = max(dp[i-1], dp[i-2] + arr[i-1])
    return dp

def mis(arr: list[z3.ArithRef], s, j) -> int:
    n = len(arr)
    o = opt_mis(arr)

    dp = z3.K(z3.IntSort(), 0)
    dp = z3.Store(dp, 1, arr[0])

    f = z3.Function('f', z3.IntSort(), z3.IntSort())

    for i in range(2, n+1):
        dp = z3.Store(dp, i, my_max([dp[i-x] + arr[i-1], dp[i-y]]))

    s.add(dp[n] == o[n])
    if s.check() == z3.sat:
        return z3.sat

s = z3.Solver()

x = z3.Int('x')
y = z3.Int('y')

n = 5
A = z3.IntVector('A', n)
s.add(x >= 0)
s.add(y >= 0)

lst = []
arrs = []
ntests = 10
for i in range(ntests):
    arr = [random.randint(0, 10) for _ in range(n)]
    arrs.append(arr)
    test = z3.Implies(A == arr, mis(arr, s, i) == z3.sat)
    lst.append(test)
tests = z3.And(lst)

s.add(
    z3.ForAll(A, tests)
)

if s.check() == z3.sat:
    print(f"x = {s.model()[x]}")
    print(f"y = {s.model()[y]}")


x = 2
y = 1


In [6]:
from z3 import *

Z = IntSort()
f = Function('f', Z, Z)
x, y, z = Ints('x y z')
A = Array('A', Z, Z)
fml = Implies(x + 2 == y, f(Store(A, x, 3)[y - 2]) == f(y - x + 1))

s = Solver()
s.add(fml)

if s.check() == sat:
    print(s.model())

[x = 0, A = K(Int, 5), y = 0]
